In [ ]:
import netCDF4 as nc
import numpy as np
from os import path

def add_max_of_max(ncinfname1, ncinfname2, ncoutfname, toexclude):
    with nc.Dataset(ncinfname1) as src1, \
         nc.Dataset(ncinfname2) as src2, \
         nc.Dataset(ncoutfname, "w") as dst:
        # copy global attributes all at once via dictionary
        dst.setncatts(src1.__dict__)
        # copy dimensions
        for name, dimension in src1.dimensions.items():
            dst.createDimension(
                name, (len(dimension) if not dimension.isunlimited() else None))
        # copy all file data except for the excluded
        for name, variable in src1.variables.items():
            if name not in toexclude:
                x = dst.createVariable(name, variable.datatype, variable.dimensions)
                # copy variable attributes all at once via dictionary
                dst[name].setncatts(src1[name].__dict__)
                dst[name][:] = src1[name][:]

        # add max_of_max
        dst_name = 'zeta_max'
        zeta_max1 = src1['zeta_max'][:]
        zeta_max2 = src2['zeta_max'][:]
        zeta_max_of_max = np.nanmax([zeta_max1, zeta_max2], axis=0)
        dst[dst_name][:] = zeta_max_of_max


basedir = "/projects/storm_surge/COMT/sbunya/runs/EmbedVEW1DtoLargeModel/NC/Florence"
ncfin = "maxele.63.nc"

cases1 = ["SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_noflows_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_swan2noswan_ERA52OWI2ERA5"]
cases2 = ["SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_nomet"]
ncfous = ["maxele_max_costal_and_runoff.63.nc"]

# toexclude = ['depth', 'zeta_max', 'time_of_zeta_max']
toexclude = ['time_of_zeta_max']

for i in range(len(cases1)):
    print(i)
    ncfname_in1 = path.join(basedir, cases1[i], 'maxele.63.nc')
    ncfname_in2 = path.join(basedir, cases2[i], 'maxele.63.nc')
    ncfname_ou = path.join(basedir, cases1[i], ncfous[i])
    add_max_of_max(ncfname_in1, ncfname_in2, ncfname_ou, toexclude)



0
length of zeta_max_of_max:  779104
